CMPT 2400: Exploratory Data Analysis Data Project
Prepared by Laura Brin, Sandra Alex & Annabell Rodriguez

loading libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split

# This makes it so we are able to see 100 rows when displaying the data
pd.set_option("display.max_rows", 100)

loading datasets

In [ ]:
pass_df=pd.read_csv("dataset/International_Report_Passengers.csv")
depart_df=pd.read_csv("dataset/International_Report_Departures.csv")

Observing Departures Dataset

This dataset comtains data on all the flights between US gateways and non-US gateways. It is a record of international flights departing US and can be used to highlight busiest airports, and peak times for flight volume

    ###Laura

Posed Problem: Flight delay propagation
International travel involves a web of interconnected airports in hundreds of countries every single day of the year. 
The ripple effect caused by a cancelled or delayed flight can cause issues with missed connections, missing baggage, carrier fines, reimbursed customers and staffing issues.

According to the Federal Aviation Administration, Delay Propagation occurs when three conditions are met simultaneously (https://aspm.faa.gov/aspmhelp/index/Delay_Propagation.html#:~:text=Delay%20propagation%20occurs%20when%20a,identified%20by%20a%20tail%20number.):
    A flight arrives late at an airport.
    A flight departs late in subsequent stages.
    A flight arrives late at the next destination.



Features

Date- in MM/DD/YYYY 

Year

Month

usg_apt_id: US Gateway Airport ID- assigned by US DOT to identify airport

usg_apt: US Gateway Airport Code- usually assigned by IATA but in absence of IAYA designation, may show FAA-assigned code

usg_wac: US Gateway World Area code- assigned by US DOT to represent a geographic territory

fg_apt_id: Foreign Gateway Airport ID-assigned by US DOT to identify an airport

fg_apt: Foreign Gateway Airport Code- usually assigned by IATA but in absence of IATA designation, may show FAA assigned code

fg_wac: Foreigh Gateway World Area Code- Assigned by US DOT to represent territory

airlineid: Airline ID assigned by US DOT to identify an air carrier




Notes: 
need to set date format correctly
year/month-numerical-any need for month to be in categorical?
need to check where more than 2 usg_apt is assigned to usg_apt_id and relabel
important pieces- US gatewayForeign Gateway and US_ world Area codeforeign world area code
year,month,
apt_ids and airlineid all as numeric-actually categorical

In [ ]:
depart_df.head(20)

In [ ]:
depart_df.describe()

In [ ]:
depart_df.shape

In [ ]:
depart_df.dtypes

    ###Annabell

This is a test

In [ ]:
print("This is a test")

    ###Sandra